# Pulse Length for H2 is 15.45 ns with Strict Partial Compilation

In [1]:
import sys
sys.path.append('../..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 6000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [10]:
N = 2
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(1, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [ ]:
circuit = uccsd.get_uccsd_circuit('H2')

In [9]:
slices = uccsd.get_uccsd_slices(circuit, granularity=1)
for slice in slices:
    if slice.parameterized:
        print('This slice is parameterized:')
    else:
        print('This slice is fixed:')
    print(slice.circuit)
    print('\n---------------------\n')

This slice is fixed:
        ┌───┐┌────────────┐┌───┐
q_0: |0>┤ X ├┤ Rx(1.5708) ├┤ X ├
        └───┘└────────────┘└───┘
q_1: |0>────────────────────────
                                

---------------------

This slice is parameterized:
        ┌────────────┐
q_0: |0>┤ Rz(6.9297) ├
        └────────────┘
q_1: |0>──────────────
                      

---------------------

This slice is fixed:
                      ┌───┐┌─────────────┐
q_0: |0>──────────────┤ X ├┤ Rx(-1.5708) ├
        ┌────────────┐└───┘└─────────────┘
q_1: |0>┤ Rx(1.5708) ├────────────────────
        └────────────┘                    

---------------------

This slice is parameterized:
                      
q_0: |0>──────────────
        ┌────────────┐
q_1: |0>┤ Rz(4.5297) ├
        └────────────┘

---------------------

This slice is fixed:
                            ┌────────────┐          
q_0: |0>────────────────────┤ Rx(1.5708) ├──■───────
        ┌─────────────┐┌───┐└────────────┘┌─┴─┐┌───┐
q_1: |0>┤ Rx(-

In [11]:
def binary_search_for_shortest_pulse_time(min_time, max_time, tolerance=1):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns."""
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 * tolerance < max_steps:  # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='Adam', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target:  # if converged, search lower half
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps / 20

In [12]:
times = []
for slice in slices:
    if not slice.parameterized:
        U = slice.unitary()
        times.append(binary_search_for_shortest_pulse_time(0.0, 10.0, tolerance=0.3))



trying total_time: 5.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00000_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :5.43e-01; Runtime: 4.4s; Iterations = 0, grads =   2.077e-01, unitary_metric = 1.00000
Error = :4.18e-02; Runtime: 5.5s; Iterations = 20, grads =   2.302e-02, unitary_metric = 1.00000
Error = :8.59e-04; Runtime: 5.8s; Iterations = 29, grads =   5.593e-03, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00000_Mar28.h5


trying total_time: 2.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00001_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators w

Error = :1.58e-01; Runtime: 4.1s; Iterations = 700, grads =   1.272e-08, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.2s; Iterations = 720, grads =   1.196e-08, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.3s; Iterations = 740, grads =   1.128e-08, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.4s; Iterations = 760, grads =   1.067e-08, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.5s; Iterations = 780, grads =   1.011e-08, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.6s; Iterations = 800, grads =   9.599e-09, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.8s; Iterations = 820, grads =   9.119e-09, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 4.9s; Iterations = 840, grads =   8.662e-09, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 5.0s; Iterations = 860, grads =   8.221e-09, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 5.1s; Iterations = 880, grads =   7.786e-09, unitary_metric = 1.00000
Error = :1.58e-01; R

Error = :1.58e-01; Runtime: 13.5s; Iterations = 2360, grads =   2.217e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 13.7s; Iterations = 2380, grads =   1.939e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 13.8s; Iterations = 2400, grads =   1.700e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 13.9s; Iterations = 2420, grads =   1.471e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.0s; Iterations = 2440, grads =   1.291e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.1s; Iterations = 2460, grads =   1.152e-11, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.2s; Iterations = 2480, grads =   9.858e-12, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.3s; Iterations = 2500, grads =   8.283e-12, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.5s; Iterations = 2520, grads =   7.155e-12, unitary_metric = 1.00000
Error = :1.58e-01; Runtime: 14.6s; Iterations = 2540, grads =   6.431e-12, unitary_metric = 1.00000


Error = :4.76e-02; Runtime: 8.4s; Iterations = 1080, grads =   8.265e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 8.5s; Iterations = 1100, grads =   7.848e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 8.7s; Iterations = 1120, grads =   7.455e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 8.8s; Iterations = 1140, grads =   7.082e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.0s; Iterations = 1160, grads =   6.730e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.1s; Iterations = 1180, grads =   6.395e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.3s; Iterations = 1200, grads =   6.078e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.4s; Iterations = 1220, grads =   5.777e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.6s; Iterations = 1240, grads =   5.491e-09, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 9.7s; Iterations = 1260, grads =   5.219e-09, unitary_metric = 1.00000
Error = :4

Error = :4.76e-02; Runtime: 21.0s; Iterations = 2740, grads =   1.212e-10, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.2s; Iterations = 2760, grads =   1.148e-10, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.3s; Iterations = 2780, grads =   1.090e-10, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.5s; Iterations = 2800, grads =   1.037e-10, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.7s; Iterations = 2820, grads =   9.880e-11, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.8s; Iterations = 2840, grads =   9.446e-11, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 21.9s; Iterations = 2860, grads =   9.057e-11, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 22.1s; Iterations = 2880, grads =   8.697e-11, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 22.2s; Iterations = 2900, grads =   8.388e-11, unitary_metric = 1.00000
Error = :4.76e-02; Runtime: 22.4s; Iterations = 2920, grads =   8.099e-11, unitary_metric = 1.00000


...

Error = :6.83e-02; Runtime: 109.0s; Iterations = 5840, grads =   5.241e-08, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 109.4s; Iterations = 5860, grads =   2.625e-07, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 109.7s; Iterations = 5880, grads =   7.521e-07, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 110.1s; Iterations = 5900, grads =   1.462e-07, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 110.5s; Iterations = 5920, grads =   6.789e-08, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 110.9s; Iterations = 5940, grads =   5.181e-08, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 111.2s; Iterations = 5960, grads =   5.275e-08, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 111.6s; Iterations = 5980, grads =   5.360e-08, unitary_metric = 1.00000
Error = :6.83e-02; Runtime: 111.9s; Iterations = 6000, grads =   5.325e-08, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00025_Mar28.h5


trying total_time: 3.75 for uni

Error = :7.66e-03; Runtime: 23.9s; Iterations = 1040, grads =   2.471e-07, unitary_metric = 1.00000
Error = :7.64e-03; Runtime: 24.3s; Iterations = 1060, grads =   2.427e-07, unitary_metric = 1.00000
Error = :7.61e-03; Runtime: 24.8s; Iterations = 1080, grads =   2.392e-07, unitary_metric = 1.00000
Error = :7.59e-03; Runtime: 25.2s; Iterations = 1100, grads =   2.364e-07, unitary_metric = 1.00000
Error = :7.56e-03; Runtime: 25.7s; Iterations = 1120, grads =   2.341e-07, unitary_metric = 1.00000
Error = :7.54e-03; Runtime: 26.2s; Iterations = 1140, grads =   2.456e-07, unitary_metric = 1.00000
Error = :7.52e-03; Runtime: 26.6s; Iterations = 1160, grads =   2.529e-07, unitary_metric = 1.00000
Error = :7.49e-03; Runtime: 27.0s; Iterations = 1180, grads =   2.314e-07, unitary_metric = 1.00000
Error = :7.47e-03; Runtime: 27.5s; Iterations = 1200, grads =   2.224e-07, unitary_metric = 1.00000
Error = :7.44e-03; Runtime: 28.0s; Iterations = 1220, grads =   2.175e-07, unitary_metric = 1.00000


Error = :6.88e-03; Runtime: 61.3s; Iterations = 2680, grads =   1.538e-08, unitary_metric = 1.00000
Error = :6.88e-03; Runtime: 61.7s; Iterations = 2700, grads =   1.508e-08, unitary_metric = 1.00000
Error = :6.88e-03; Runtime: 62.2s; Iterations = 2720, grads =   1.455e-08, unitary_metric = 1.00000
Error = :6.88e-03; Runtime: 62.7s; Iterations = 2740, grads =   1.415e-08, unitary_metric = 1.00000
Error = :6.88e-03; Runtime: 63.1s; Iterations = 2760, grads =   1.377e-08, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 63.5s; Iterations = 2780, grads =   1.346e-08, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 64.0s; Iterations = 2800, grads =   1.313e-08, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 64.5s; Iterations = 2820, grads =   1.281e-08, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 64.9s; Iterations = 2840, grads =   1.252e-08, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 65.4s; Iterations = 2860, grads =   1.597e-08, unitary_metric = 1.00000


Error = :6.92e-03; Runtime: 98.6s; Iterations = 4320, grads =   7.171e-08, unitary_metric = 1.00000
Error = :6.91e-03; Runtime: 99.1s; Iterations = 4340, grads =   9.374e-08, unitary_metric = 1.00000
Error = :6.90e-03; Runtime: 99.5s; Iterations = 4360, grads =   4.515e-06, unitary_metric = 1.00000
Error = :6.89e-03; Runtime: 99.9s; Iterations = 4380, grads =   6.700e-06, unitary_metric = 1.00000
Error = :6.89e-03; Runtime: 100.4s; Iterations = 4400, grads =   9.242e-07, unitary_metric = 1.00000
Error = :6.88e-03; Runtime: 100.9s; Iterations = 4420, grads =   1.732e-07, unitary_metric = 1.00000
Error = :6.87e-03; Runtime: 101.3s; Iterations = 4440, grads =   7.157e-08, unitary_metric = 1.00000
Error = :6.86e-03; Runtime: 101.8s; Iterations = 4460, grads =   6.223e-08, unitary_metric = 1.00000
Error = :6.85e-03; Runtime: 102.2s; Iterations = 4480, grads =   5.930e-08, unitary_metric = 1.00000
Error = :6.84e-03; Runtime: 102.6s; Iterations = 4500, grads =   5.519e-08, unitary_metric = 1.

Error = :6.70e-03; Runtime: 136.0s; Iterations = 5960, grads =   5.870e-09, unitary_metric = 1.00000
Error = :6.70e-03; Runtime: 136.4s; Iterations = 5980, grads =   3.245e-07, unitary_metric = 1.00000
Error = :6.70e-03; Runtime: 136.9s; Iterations = 6000, grads =   1.631e-08, unitary_metric = 1.00000
data saved at: /project/ftchong/qoc/pranav/00027_Mar28.h5


trying total_time: 3.4 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00028_Mar28.h5
Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.46e-01; Runtime: 0.1s; Iterations = 0, grads =   7.774e-02, unitary_metric = 1.00000
Error = :5.49e-01; Runtime: 0.8s; Iterations = 20, grads =   1.454e-02, unitary_metric = 1.00000
Error = 

In [13]:
print(times)
print(sum(times))

[1.05, 1.05, 3.55, 4.8, 3.4]
13.85


In [14]:
parametrized_times = [np.round(util.circuitutil.get_max_pulse_time(
    slice.circuit), 4) for slice in slices if slice.parameterized]
print(sum(parametrized_times))

1.6


In [15]:
print(sum(parametrized_times) + sum(times))

15.45
